# Chainerで多層パーセプトロンを作ろう
## MNISTデータで手書き文字認識

- このファイルは以下のGitHubディレクトリにあるtrain_mnist.pyをipython notebookにペタペタ貼ったものです。
https://github.com/pfnet/chainer/tree/master/examples/mnist
- [Chainer@GitHub](https://github.com/pfnet/chainer.git)をクローンしてきて、chainer/examples/mnist/の中においても使えます。

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

In [ ]:
from chainer import Variable, Chain, optimizers
import chainer.functions as F
import chainer.links as L

## データの読み込み

In [ ]:
import data

In [ ]:
mnist = data.load_mnist_data()

In [ ]:
mnist

In [ ]:
mnist['data'].max()

In [ ]:
x_all = mnist['data'].astype(np.float32) / 255
y_all = mnist['target'].astype(np.int32)

In [ ]:
x_train, x_test = np.split(x_all, [60000])
y_train, y_test = np.split(y_all, [60000])

## モデルを定義

In [ ]:
class MLP(Chain):
    def __init__(self):
        super(MLP, self).__init__(
            l1 = L.Linear(784, 100),
            l2 = L.Linear(100, 100),
            l3 = L.Linear(100, 10),
        )
        
    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        y = self.l3(h2)
        return y

損失関数と分類精度の計算をするために、MLP chainの上にclassifier chainを作る。

In [ ]:
class MyClassifier(Chain):
    """
    Compute accuracy and loss.
    
    Returns:
        loss
    """
    
    def __init__(self, predictor):
        super(MyClassifier, self).__init__(
            predictor=predictor
        )
    
    def __call__(self, x, t):
        y = self.predictor(x)
        self.loss = F.softmax_cross_entropy(y, t)
        self.accuracy = F.accuracy(y, t)
        return self.loss

今、定義したMyClassifierクラスと似たクラスが、chainer.links.Classifierに定義されているので、通常はそっちをつかうべし。

In [ ]:
model = MyClassifier(MLP())
#model = L.Classifier(MLP())    # <- 本来ならこっちの方が良い

In [ ]:
optimizer = optimizers.SGD()

In [ ]:
optimizer.setup(model)

### 書き方１

In [ ]:
batchsize = 100
datasize = 60000

In [ ]:
for epoch in range(20):
    print('epoch %d' % epoch)
    indices = np.random.permutation(datasize)
    for i in range(0, datasize, batchsize):
        x = Variable(x_train[indices[i : i + batchsize]])
        t = Variable(y_train[indices[i : i + batchsize]])
        
        optimizer.update(model, x, t)

### 書き方２

In [ ]:
batchsize = 100
datasize = 60000

In [ ]:
for epoch in range(20):
    print('epoch %d' % epoch)
    
    # Compute test/validation error
    sum_loss, sum_accuracy = 0, 0
    for i in range(0, 10000, batchsize):
        x = Variable(x_test[i : i + batchsize])
        t = Variable(y_test[i : i + batchsize])
        loss = model(x, t)
        sum_loss += loss.data * batchsize
        sum_accuracy += model.accuracy.data * batchsize

    mean_loss = sum_loss / 10000
    mean_accuracy = sum_accuracy / 10000

    print("mean loss: %.5f\t mean accuracy: %.5f" % (mean_loss, mean_accuracy))    
    
    indices = np.random.permutation(datasize)
    for i in range(0, datasize, batchsize):
        x = Variable(x_train[indices[i : i + batchsize]])
        t = Variable(y_train[indices[i : i + batchsize]])
        
        model.zerograds()
        loss = model(x, t)
        loss.backward()
                
        optimizer.update()